In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

In [3]:
v = tf.Variable(0.0)

In [4]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [5]:
(v + 1).numpy()

1.0

In [6]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [7]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [8]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [9]:
v.read_value()

<tf.Tensor: id=17, shape=(), dtype=float32, numpy=6.0>

In [10]:
w = tf.Variable(1.0)
with tf.GradientTape() as t:
    loss = w * w

In [11]:
grad = t.gradient(loss, w)

In [12]:
grad

<tf.Tensor: id=31, shape=(), dtype=float32, numpy=2.0>

In [13]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [14]:
dloss_dw = t.gradient(loss, w) 

In [15]:
dloss_dw

<tf.Tensor: id=37, shape=(), dtype=float32, numpy=6.0>

In [16]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y 

In [17]:
dy_dw = t.gradient(y, w) 

In [18]:
print(dy_dw)

tf.Tensor(6.0, shape=(), dtype=float32)


In [19]:
dz_dw = t.gradient(z, w)

In [20]:
dz_dw

<tf.Tensor: id=51, shape=(), dtype=float32, numpy=108.0>

In [21]:
(train_image, train_label), _ = tf.keras.datasets.mnist.load_data()

In [22]:
train_image.shape,train_image.dtype

((60000, 28, 28), dtype('uint8'))

In [23]:
train_image = tf.expand_dims(train_image, -1)

In [24]:
train_image.shape,train_image.dtype

(TensorShape([60000, 28, 28, 1]), tf.uint8)

In [25]:
train_image = tf.cast(train_image/255, tf.float32)

In [26]:
train_image.shape,train_image.dtype

(TensorShape([60000, 28, 28, 1]), tf.float32)

In [27]:
train_label = tf.cast(train_label, tf.int64)

In [28]:
train_label.dtype

tf.int64

In [29]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))

In [30]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [31]:
dataset = dataset.shuffle(10000).batch(32)

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [34]:
optimizer = tf.keras.optimizers.Adam()

In [35]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
features, labels = next(iter(dataset))

In [37]:
features.shape, labels.shape

(TensorShape([32, 28, 28, 1]), TensorShape([32]))

In [38]:
predictions = model(features)

In [39]:
predictions.shape

TensorShape([32, 10])

In [40]:
tf.argmax(predictions, axis=1)

<tf.Tensor: id=199, shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

In [41]:
labels

<tf.Tensor: id=181, shape=(32,), dtype=int64, numpy=
array([3, 9, 7, 6, 4, 6, 8, 5, 2, 3, 6, 1, 2, 5, 5, 4, 0, 1, 4, 1, 0, 8,
       6, 7, 5, 7, 8, 6, 1, 1, 0, 1])>

In [42]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [43]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        loss_step = loss(model, images, labels)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [44]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
#         print('Epoch{} is finished'.format(epoch))
        print('Epoch%d is finished' % epoch)

In [45]:
train()

Epoch0 is finished
Epoch1 is finished
Epoch2 is finished
Epoch3 is finished
Epoch4 is finished
Epoch5 is finished
Epoch6 is finished
Epoch7 is finished
Epoch8 is finished
Epoch9 is finished
